In [2]:
activities_list = """chop an onion
fill a bucket in a small sink
folding piece of cloth
freeze pies
make a steak
make a strawberry slushie
mowing the lawn
spraying for bugs""".replace(" ", "_").replace("-", "_").replace("'", "_").split("\n")
activities = set(activities_list)

## Load list of available objects

In [53]:
# Are there any categories that are missing from the table?
import pathlib, glob, json, os
from collections import defaultdict
available_scenes = {"Rs_int", "Beechwood_0_int", "Wainscott_0_int", "Ihlen_0_int", "Merom_1_int"}
providers = {}
objects_by_category = defaultdict(list)
object_lists = glob.glob(r"D:\ig_pipeline\cad\objects\*\artifacts\object_list.json")
scene_object_lists = [os.path.join(r"D:\ig_pipeline\cad\scenes", scene, r"artifacts\object_list.json") for scene in available_scenes]
for olf in object_lists + scene_object_lists:
    dirname = pathlib.Path(olf).parts[-4] + "/" + pathlib.Path(olf).parts[-3]
    with open(olf, "r") as f:
        ol = json.load(f)
    if "scenes" not in olf:
        pass
        # Some filters for object files
        # if not ol["success"]:
        #     continue
        # if len(ol["provided_objects"]) > 10:
        #     continue
    for obj in ol["provided_objects"]:
        providers[obj] = dirname
        objects_by_category[obj.split("-")[0]].append(obj)

In [54]:
len(providers)

3080

In [55]:
provided_categories = {x.split("-")[0] for x in providers.keys()}

## Load list of task-required synsets

In [32]:
import json, csv
from nltk.corpus import wordnet as wn
import networkx as nx

# Build the legit-synset graph
G = nx.DiGraph()
G.add_nodes_from(x.name() for x in wn.all_synsets())
for parent in wn.all_synsets():
    for child in parent.hyponyms():
        G.add_edge(parent.name(), child.name())
        
# Add the illegit-synset graph
with open(r"C:\Users\Cem\Downloads\custom_synsets.csv") as f:
    reader = csv.DictReader(f)
    for row in reader:
        child = row["custom_synset"].strip()
        parent = wn.synset(row["hypernyms"].strip()).name()
        assert parent in G.nodes, "Could not find " + parent
        G.add_edge(parent, child)
        
legit_synsets = set(G.nodes)

In [33]:
def canonicalize(s):
    try:
        return wn.synset(s).name()
    except:
        return s

In [34]:
with open(r"D:\ig_pipeline\metadata\b200_objects.json", "r") as f:
    obj_mapping = json.load(f)
    task_required_synsets_by_activity = {k: [canonicalize(x) for x in obj_mapping[k] if x != "agent.n.01"] for k in activities}
task_required_synsets = {x for objs in task_required_synsets_by_activity.values() for x in objs}

In [35]:
len(task_required_synsets)

38

In [36]:
illegal_task_required_synsets = {x for x in task_required_synsets - legit_synsets}
print(illegal_task_required_synsets)

set()


In [37]:
legit_task_required_synsets = (task_required_synsets & legit_synsets)

## Load list of synsets in our dataset

In [38]:
import csv
# Get the category - synset mapping
pairs = {}
synset_to_cat = defaultdict(list)
with open(r"D:\ig_pipeline\metadata\category_mapping.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        category = row["category"].strip()
        synset = row["synset"].strip()
        if not synset or not category:
            print(f"Skipping problematic row: {row}")
            continue
        if category not in provided_categories:
            continue
        synset = canonicalize(synset)
        pairs[category] = synset
        synset_to_cat[synset].append(category)
found_synsets = set(pairs.values())
found_categories = set(pairs.keys())

In [39]:
# What are the categories that for some reason are not added to the spreadsheet?
provided_categories - found_categories

{'acetone',
 'acid',
 'adhseive_strip',
 'air_filter',
 'air_freshner',
 'alfredo_sauce',
 'algae',
 'aluminium_foil',
 'ammonia',
 'applesauce',
 'arepa',
 'ash',
 'beans',
 'bin',
 'bird_feeder',
 'bleach',
 'blow_dryer',
 'bowl_of_salad',
 'broccolini',
 'cactus',
 'calendar',
 'can',
 'caraway_seeds',
 'cash_box',
 'cashew',
 'casserole_dish',
 'catalog',
 'caulk',
 'cd',
 'centerpiece',
 'chainsaw',
 'chalkboard',
 'champagne',
 'charger',
 'cheesecake',
 'chicken_bouillon',
 'chicken_tender',
 'chicken_wire',
 'chickpeas',
 'chips',
 'chocolate',
 'chocolate_biscuit',
 'chocolate_spread',
 'chopblock',
 'chopsticks',
 'cilantro',
 'cioppino',
 'clam',
 'cleaner',
 'cocktail',
 'copper_wire',
 'doll',
 'duster',
 'dvd',
 'fork',
 'ice_tray',
 'icicle_lights',
 'kabob',
 'key',
 'kitchen_scales',
 'label',
 'lacquer',
 'latte',
 'lavender_oil',
 'leaf',
 'leaf_blower',
 'leather_boots',
 'leek',
 'left_bookend',
 'leggings',
 'lens',
 'lentils',
 'leotard',
 'license_plate',
 'lill

In [40]:
found_invalid_synsets = {x for x in found_synsets if x not in legit_synsets and canonicalize(x) not in legit_synsets}
print("\n".join(found_invalid_synsets))

In [41]:
for act in activities_list:
    required = task_required_synsets_by_activity[act]
    found = set()
    for s in required:
        if s not in legit_synsets:
            continue
        child_synsets = [s] + list(nx.descendants(G, s))
        for cs in child_synsets:
            if cs in found_synsets:
                found.add(s)
                break
                
    missing = set(required) - found
    # print("1" if not missing else 0) 
    print(", ".join(sorted(missing)))
    





black_pepper.n.02
agave.n.01, lemon_juice.n.01
lawn.n.01
lawn.n.01


In [42]:
# How many of the required synsets exist:
from collections import defaultdict
found_task_required_synsets = set()
found_task_required_objs_by_synset = defaultdict(set)
for s in legit_task_required_synsets:
    # Get the tree of the synset
    child_synsets = [s] + list(nx.descendants(G, s))
    for cs in child_synsets:
        if cs in found_synsets:
            found_task_required_synsets.add(s)  # Only add the sought-after synset, not children
            
            s_cats = synset_to_cat[cs]
            s_objs = {obj for cat in s_cats for obj in objects_by_category[cat]}
            found_task_required_objs_by_synset[s].update(s_objs)

found_task_required_objects = {cat for cats in found_task_required_objs_by_synset.values() for cat in cats}
not_found_task_required_synsets = legit_task_required_synsets - found_task_required_synsets
print(len(found_task_required_synsets), "found")
print(len(not_found_task_required_synsets), "not found")
print(not_found_task_required_synsets)

34 found
4 not found
{'agave.n.01', 'black_pepper.n.02', 'lawn.n.01', 'lemon_juice.n.01'}


In [43]:
# What synsets match the most crap?
sorted([(-len(objs), s) for s, objs in found_task_required_objs_by_synset.items()])

[(-245, 'container.n.01'),
 (-90, 'cabinet.n.01'),
 (-47, 'floor.n.01'),
 (-23, 'box.n.01'),
 (-22, 'piece_of_cloth.n.01'),
 (-12, 'bottle.n.01'),
 (-12, 'oven.n.01'),
 (-11, 'cup.n.01'),
 (-10, 'garlic.n.02'),
 (-10, 'knife.n.01'),
 (-9, 'ice.n.01'),
 (-9, 'onion.n.03'),
 (-9, 'sink.n.01'),
 (-8, 'glass.n.02'),
 (-8, 'pie.n.01'),
 (-8, 'stove.n.01'),
 (-7, 'chopping_board.n.01'),
 (-6, 'dryer.n.01'),
 (-6, 'electric_refrigerator.n.01'),
 (-5, 'grass.n.01'),
 (-4, 'bucket.n.01'),
 (-3, 'blender.n.01'),
 (-3, 'olive_oil.n.01'),
 (-3, 'pan.n.01'),
 (-3, 'strawberry.n.01'),
 (-2, 'lawn_mower.n.01'),
 (-1, 'atomizer.n.01'),
 (-1, 'countertop.n.01'),
 (-1, 'insectifuge.n.01'),
 (-1, 'lemon_juice__bottle.n.01'),
 (-1, 'salt.n.02'),
 (-1, 'smoothie.n.02'),
 (-1, 'steak.n.01'),
 (-1, 'water.n.06')]

In [44]:
# How many objects are task-relevant?
print(f"{len(found_task_required_objects)} relevant objects")
print(found_task_required_objects)

523 relevant objects
{'bottom_cabinet_no_top-vdedzt', 'top_cabinet-dmwxyl', 'salt-iomwtn', 'stove-igwqpj', 'bottom_cabinet_no_top-ufhpbn', 'bottom_cabinet_no_top-exgntg', 'bottom_cabinet-pzjecb', 'spoon-aeubta', 'vase-hliauj', 'floors-ucjdgj', 'bottom_cabinet-hnxyev', 'bottom_cabinet-rvpunw', 'bug_repellent-qqztry', 'floors-zvyodb', 'bottom_cabinet-xpdbgf', 'cruet-njqmqv', 'carving_knife-awvoox', 'floors-zijaqk', 'kettle-johiyp', 'vidalia_onion-yxwjgn', 'bottom_cabinet-qacthv', 'floors-dclqap', 'truck-oxbgfh', 'glass-ewgotr', 'dutch_oven-tyxwap', 'bowl-mspdar', 'trash_can-eahqyq', 'electric_kettle-hkdsla', 'cup-ckkwmj', 'floors-kxqtap', 'floors-mvnkva', 'bathtub-oobbwn', 'top_cabinet-fqhdne', 'smoothie-weothj', 'gift_bag-wvhmww', 'bottom_cabinet-nrlayx', 'onion-ndwwog', 'lemon_juice_bottle-vsjter', 'towel-pingqd', 'jar-qzwijs', 'glass-slscza', 'fridge-hzgqdn', 'top_cabinet-yegcau', 'grass_patch-kqhokv', 'ice_cube-iiimgo', 'ice_cube-xupugj', 'cup-nzdsmu', 'trash_can-hqdnjz', 'floors-qhf

In [45]:
# Get a subsampled version of this such that there is at least 1 thing matching every synset
G2 = nx.Graph()
for s, objs in found_task_required_objs_by_synset.items():
    for obj in objs:
        G2.add_edge(s, obj)
assert nx.bipartite.is_bipartite(G2)
top_set = set(found_task_required_objs_by_synset.keys())
matching = nx.bipartite.maximum_matching(G2, top_set)
cover = nx.bipartite.to_vertex_cover(G2, matching, top_set) & top_set   # TODO: THIS IS INCORRECT!!!
# assert not cover - top_set, str(cover - top_set)
subsampled_found_task_required_objects = set(matching.values()) - top_set

## Load list of objects required by our scenes

In [46]:
# Required by scene
import os
objects_needed_by_scene = {}
for scene, scene_json in zip(available_scenes, scene_object_lists):
    with open(scene_json, "r") as f:
        scene_ol = json.load(f)
    objects_needed_by_scene[scene] = scene_ol["needed_objects"]
all_scene_required_objects = {obj for objs in objects_needed_by_scene.values() for obj in objs}

## Get the combined scene / task object list

In [47]:
b66_required_objects = {
"alarm-dkwmmf",
"apple-qusmpx",
"backpack-gvbiwl",
"bagel-zlxkry",
"ball-nofzds",
"basket-clziqw",
"basket-pgwdlg",
"basket-pqzyxx",
"basket-xjftlf",
"bath_towel-lyqgnf",
"beer-phdimo",
"bow-fhchql",
"bowl-ajzltc",
"bowl-dalyim",
"bowl-eipwho",
"bowl-fedafr",
"bowl-fiarri",
"bowl-fwdfeg",
"bowl-jfvjep",
"bracelet-thqqmo",
"broom-tpyvbt",
"bucket-bdhvnt",
"bucket-lsmlzi",
"bucket-wlilma",
"calculator-kwmmty",
"candle-bhaqam",
"carafe-ocjcgp",
"carton-causya",
"catsup-hvxkso",
"cauldron-zndohl",
"chaise_longue-ajjxcd",
"chaise_longue-xmjcle",
"cheese-hwxeto",
"chest-gbdzls",
"chip-qstxpj",
"christmas_tree_decorated-wakmhn",
"cologne-lyipur",
"container_date-qqddba",
"cookie-dajnau",
"cookie-ggpang",
"cookie-oyhoth",
"cup-ckkwmj",
"cup-nzdsmu",
"cup-rypdvd",
"cup-ykuftq",
"detergent-gkpmii",
"detergent-qjkmhq",
"dishtowel-dtfspn",
"dishtowel-ltydgg",
"dishtowel-qvzbjx",
"document-qkrlsi",
"dustpan-espzwv",
"envelope-urcigc",
"facsimile-mcqqhy",
"fish-qofuoa",
"floor_lamp-jqsuky",
"floor_lamp-lqdmym",
"folder-piezyg",
"folder-poqpux",
"frying_pan-sfbdjn",
"grill-ddlqye",
"gym_shoe-kmcbym",
"gym_shoe-wfmrpw",
"hairbrush-bdylnb",
"hamburger-fyswch",
"hammer-bbzlel",
"hand_towel-jaubic",
"hardback-nubbfj",
"highchair-pahqnp",
"highlighter-tcsmqv",
"jeans-nmvvil",
"kettle-bzisss",
"kiwi-khupqq",
"milk-xbvdpc",
"modem-axqxsv",
"mousetrap-mwfwsv",
"mug-ppzttc",
"newspaper-ukamcl",
"notebook-aanuhi",
"notebook-svisnb",
"oatmeal-zkggxm",
"olive-mhnfdr",
"olive_oil-cqycjk",
"olive_oil-luikop",
"painting-ggloob",
"paper_bag-bzsxgw",
"paper_towel-aclaus",
"parsley-kkgkkd",
"pasta-gwqhko",
"pen-zmimhb",
"pencil-xuanpj",
"plate-iawoof",
"plate-ivbrtz",
"plate-nbuspz",
"plate-qbxfmv",
"plywood-fkmkqa",
"pool-xixlzr",
"pop-bqretx",
"pop-drqbiy",
"pop-ghxeqz",
"pop-mrrozu",
"pop-ubazru",
"pop-vszbvb",
"pop-xlyult",
"pop-xoldze",
"potato-bpwohr",
"pretzel-rmutwn",
"printer-oyqbtq",
"pumpkin-apitmi",
"rag-dgzewy",
"rag-zmmbpz",
"rib-nzbnqp",
"ribbon-puwdwq",
"salad-dhdhul",
"sandal-chgcrm",
"sandal-uhzfis",
"saucepan-fsinsu",
"saw-laysom",
"scanner-juzkjp",
"scraper-icqmje",
"scrub_brush-hsejyi",
"shampoo-dvrzmy",
"shampoo-jccyom",
"shampoo-zoatjo",
"sheet-czdhwu",
"soap-ozifwa",
"sock-rhgweu",
"spaghetti_sauce-zahyds",
"spoon-gpiftn",
"spoon-prbhag",
"sticky_note-tghqep",
"sugar_jar-rfzqei",
"sushi-akpnxn",
"sweater-ekcjci",
"sweater-eknmzl",
"teapot-foaehs",
"tile-kuombe",
"tile-vowkci",
"toothbrush-qbkrzg",
"toothbrush-vkrjps",
"toothpaste-vsclsj",
"toy-aihmqy",
"toy-cobfpg",
"toy-fnueyn",
"toy-tjchdg",
"toy-vcmcoi",
"toy-zuntus",
"trash_can-aefcem",
"trash_can-pdmzhv",
"trash_can-zotrbg",
"tray-spopfj",
"tray-xzcnjq",
"underwear-uwzqxm",
"vacuum-bdmsbr",
"vacuum-wikhik",
"walker-txpihh",
"walker-ykjauh",
"wine_bottle-fzuiwy",
"wrapped_gift-wpmksn",
"wreath-rkrxno",
"armchair-bslhmj",
"armchair-qplklw",
"armchair-tcxiue",
"armchair-vzhxuf",
"armchair-xpcheg",
"basket-clziqw",
"basket-wdsnjy",
"bathtub-fdjykf",
"bed-smmmaf",
"bed-zrumze",
"bottom_cabinet-bamfsz",
"bottom_cabinet-dajebq",
"bottom_cabinet-immwzb",
"bottom_cabinet-jhymlr",
"bottom_cabinet-jrhgeu",
"bottom_cabinet-lwjdmj",
"bottom_cabinet-nddvba",
"bottom_cabinet-qacthv",
"bottom_cabinet-rvpunw",
"bottom_cabinet-slgzfc",
"bottom_cabinet_no_top-bmsclc",
"bottom_cabinet_no_top-pluwfl",
"bottom_cabinet_no_top-qohxjq",
"bottom_cabinet_no_top-qudfwe",
"bottom_cabinet_no_top-spojpj",
"bottom_cabinet_no_top-ufhpbn",
"breakfast_table-dnsjnv",
"breakfast_table-skczfi",
"breakfast_table-uhrsex",
"breakfast_table-zypvuv",
"burner-pmntxh",
"carpet-ctclvd",
"ceilings-ayfjzo",
"ceilings-cbjzgq",
"ceilings-ckxrok",
"ceilings-dashlx",
"ceilings-dslvvl",
"ceilings-dulwkz",
"ceilings-eanlkl",
"ceilings-edrgnz",
"ceilings-ekhjwh",
"ceilings-gbfubh",
"ceilings-gdodun",
"ceilings-gjwnre",
"ceilings-gvaqgm",
"ceilings-gztsev",
"ceilings-hzdavr",
"ceilings-ibqleu",
"ceilings-jfsbfp",
"ceilings-kjpzku",
"ceilings-lfwahr",
"ceilings-lmlzab",
"ceilings-ndlpeo",
"ceilings-odgyya",
"ceilings-oetibq",
"ceilings-ofefox",
"ceilings-pcatii",
"ceilings-phrrml",
"ceilings-qquywv",
"ceilings-rhfker",
"ceilings-rlsnwq",
"ceilings-rnaypg",
"ceilings-rxnnfd",
"ceilings-taubkv",
"ceilings-trhqld",
"ceilings-uahaxk",
"ceilings-vjiuwu",
"ceilings-vnstqh",
"ceilings-vubcbl",
"ceilings-vwwzdu",
"ceilings-vyvqsb",
"ceilings-wvqyit",
"ceilings-wxbmon",
"ceilings-xqauyk",
"ceilings-xusryb",
"ceilings-xzgtpt",
"ceilings-yimfml",
"ceilings-zfnsml",
"ceilings-zlhppj",
"chest-fwstpx",
"coffee_maker-pyttso",
"coffee_table-fqluyq",
"coffee_table-gpkbiw",
"coffee_table-qlmqyy",
"console_table-emeeke",
"countertop-tpuwys",
"dishwasher-dngvvi",
"dishwasher-xlmier",
"dishwasher-znfvmj",
"door-ktydvs",
"door-lvgliq",
"door-ohagsq",
"dryer-xsuyua",
"dryer-zlmnfg",
"floor_lamp-jcbvfi",
"floor_lamp-jqsuky",
"floor_lamp-vdxlda",
"floors-aafddg",
"floors-avvnuv",
"floors-bryjan",
"floors-cborev",
"floors-dclqap",
"floors-dggxhh",
"floors-ekqfhr",
"floors-gjemfr",
"floors-gjpvfd",
"floors-gupykd",
"floors-gwawvk",
"floors-hcqtge",
"floors-ibjsgt",
"floors-ideoae",
"floors-ifmioj",
"floors-jbiwrx",
"floors-jniagb",
"floors-kqgjaj",
"floors-kwnype",
"floors-kxqtap",
"floors-ljbnpd",
"floors-lwmene",
"floors-meryyw",
"floors-mknpoc",
"floors-mowtdm",
"floors-mvnkva",
"floors-nuacoz",
"floors-obpiof",
"floors-pmlwwl",
"floors-ptwlei",
"floors-qhfsxw",
"floors-qrzmkt",
"floors-qsesig",
"floors-qtdpcm",
"floors-rmwqgp",
"floors-rqcmwl",
"floors-sktjer",
"floors-sqtkxe",
"floors-ssjixw",
"floors-suqrjf",
"floors-tcsswg",
"floors-ucjdgj",
"floors-umeasc",
"floors-xzlkei",
"floors-zijaqk",
"floors-znzabl",
"floors-zpfzjw",
"floors-zvyodb",
"fridge-dszchb",
"fridge-xyejdx",
"grandfather_clock-veyqcp",
"laptop-nvulcs",
"loudspeaker-bmpdyv",
"microwave-abzvij",
"microwave-bfbeeb",
"mirror-pevdqu",
"oven-fexqbj",
"oven-wuinhm",
"piano-bnxcvw",
"picture-cltbty",
"picture-etixod",
"picture-fanvpf",
"picture-fhkzlm",
"picture-gwricv",
"picture-hhxttu",
"picture-jpfyrq",
"picture-lucjyq",
"picture-qavxsz",
"picture-qjkajj",
"picture-qtvjzk",
"picture-rciuob",
"picture-szpupz",
"picture-tlwjpv",
"picture-weqggl",
"picture-wfdvzv",
"picture-yrgaal",
"picture-ytxhyl",
"picture-zsirgc",
"pillow-pjqqmb",
"pot_plant-jatssq",
"pot_plant-kxmvco",
"pot_plant-udqjui",
"rail_fence-qmsnld",
"range_hood-iqbpie",
"shelf-njwsoa",
"shelf-owvfik",
"shelf-vgzdul",
"shower-invgma",
"sink-czyfhq",
"sink-ksecxq",
"sink-xiybkb",
"sink-zexzrc",
"sofa-mnfbbh",
"sofa-qnnwfx",
"sofa-uixwiv",
"sofa-xhxdqf",
"speaker_system-snbvop",
"standing_tv-udotid",
"stool-miftfy",
"stool-xcmniq",
"stool-ycfbsd",
"stove-igwqpj",
"stove-rgpphy",
"straight_chair-amgwaw",
"straight_chair-dmcixv",
"straight_chair-eospnr",
"straight_chair-psoizi",
"straight_chair-vkgbbl",
"swivel_chair-qtqitn",
"table_lamp-xbfgjc",
"toilet-kfmkbm",
"toilet-vtqdev",
"toilet-wusctd",
"top_cabinet-dmwxyl",
"top_cabinet-eobsmt",
"top_cabinet-fqhdne",
"top_cabinet-jvdbxh",
"top_cabinet-lsyzkh",
"towel_rack-kqrmrh",
"trash_can-zotrbg",
"wall_mounted_tv-ylvjhb",
"walls-adqmpz",
"walls-afdfgb",
"walls-aigrtq",
"walls-aquzfl",
"walls-axhynp",
"walls-ayjzsb",
"walls-bbqdud",
"walls-bcjmdz",
"walls-bdxwvm",
"walls-bntgiu",
"walls-bqdrep",
"walls-btxdrd",
"walls-bvcaju",
"walls-bvtxbv",
"walls-cesukw",
"walls-cusskx",
"walls-cyflyo",
"walls-dbcxft",
"walls-dekben",
"walls-dfnqnk",
"walls-dondsu",
"walls-dthtvd",
"walls-dzlwbx",
"walls-ejzliv",
"walls-enroos",
"walls-exiopd",
"walls-eyalcx",
"walls-fsaxgs",
"walls-fxibmi",
"walls-gfxhgi",
"walls-gmdnub",
"walls-gtaxam",
"walls-gxvfxw",
"walls-gxynpx",
"walls-hfwelb",
"walls-hmmmqw",
"walls-hnmcoo",
"walls-hnqegn",
"walls-hosciu",
"walls-hqpeoy",
"walls-hsudbq",
"walls-hxakof",
"walls-hyizxy",
"walls-icgqdt",
"walls-icgyhn",
"walls-ifhuaw",
"walls-insggh",
"walls-ioxzum",
"walls-ipzkkp",
"walls-iqbzlu",
"walls-iwgkkg",
"walls-iwipoh",
"walls-jukzkm",
"walls-jvltxt",
"walls-kabquv",
"walls-kvmewy",
"walls-ljngkt",
"walls-llgahv",
"walls-lmdouk",
"walls-mqmdyl",
"walls-ngfxil",
"walls-nmzbqa",
"walls-nthwqa",
"walls-nvtvxi",
"walls-nxxtrq",
"walls-nyvztg",
"walls-odbsjx",
"walls-pefrto",
"walls-pivmuk",
"walls-pmfadr",
"walls-pmqiht",
"walls-prkntu",
"walls-pziwqm",
"walls-qannys",
"walls-qbpprw",
"walls-qcqpyi",
"walls-qisxmx",
"walls-qyejch",
"walls-radpsr",
"walls-rkueec",
"walls-rvvbvi",
"walls-sdxbli",
"walls-snokhn",
"walls-szxayl",
"walls-tjfjwe",
"walls-tmpzcd",
"walls-tsyvvt",
"walls-twxspa",
"walls-ucsylc",
"walls-ugemkf",
"walls-uishdq",
"walls-umjlad",
"walls-uwsqqu",
"walls-vdzmpj",
"walls-vrkkum",
"walls-wcekyw",
"walls-wiihpm",
"walls-wxyhis",
"walls-wycsdd",
"walls-xamxbe",
"walls-xkdbsb",
"walls-xlgydu",
"walls-xrmvpg",
"walls-xwrjsn",
"walls-yeodqe",
"walls-yjbruo",
"walls-yrcjvs",
"walls-ysxqfc",
"walls-zbggqd",
"walls-zlszjq",
"walls-zoonsb",
"walls-zqimeh",
"walls-zssquq",
"washer-omeuop",
"window-ithrgo",
"window-ulnafj",
    }

In [59]:
print(all_scene_required_objects - b66_required_objects)
print(b66_required_objects - set(providers.keys()))

set()
{'floors-umeasc'}


In [56]:
all_required_objects = b66_required_objects
# all_required_objects = subsampled_found_task_required_objects | all_scene_required_objects | {obj for obj, provider in providers.items() if "legacy" in provider}
print(f"{len(all_scene_required_objects)} objects required by scenes")
print(f"{len(subsampled_found_task_required_objects)} objects required by tasks")
print(f"{len(all_required_objects)} objects required total")

333 objects required by scenes
34 objects required by tasks
491 objects required total


In [60]:
# Get the right set of files
sources = sorted({providers[x] for x in all_required_objects if x in providers})
print("\n".join([f'"{x}",' for x in sources]))

"objects/bratwurst-lt",
"objects/care_products-gt",
"objects/dog_food-df",
"objects/legacy_alarm-dkwmmf",
"objects/legacy_apple-qusmpx",
"objects/legacy_armchair-bslhmj",
"objects/legacy_armchair-qplklw",
"objects/legacy_armchair-tcxiue",
"objects/legacy_armchair-vzhxuf",
"objects/legacy_armchair-xpcheg",
"objects/legacy_backpack-gvbiwl",
"objects/legacy_bagel-zlxkry",
"objects/legacy_ball-nofzds",
"objects/legacy_basket-clziqw",
"objects/legacy_basket-pgwdlg",
"objects/legacy_basket-pqzyxx",
"objects/legacy_basket-wdsnjy",
"objects/legacy_basket-xjftlf",
"objects/legacy_bath_towel-lyqgnf",
"objects/legacy_bathtub-fdjykf",
"objects/legacy_bed-smmmaf",
"objects/legacy_bed-zrumze",
"objects/legacy_beer-phdimo",
"objects/legacy_bottom_cabinet-bamfsz",
"objects/legacy_bottom_cabinet-dajebq",
"objects/legacy_bottom_cabinet-immwzb",
"objects/legacy_bottom_cabinet-jhymlr",
"objects/legacy_bottom_cabinet-jrhgeu",
"objects/legacy_bottom_cabinet-lwjdmj",
"objects/legacy_bottom_cabinet-nddvba",
"

In [61]:
print(len(sources))

286


## Check how many of the objects have already been successfully exported

In [24]:
successful = 0
tried = 0
for s in sources:
    ems = os.path.join(r"D:\ig_pipeline\cad", s, r"artifacts\export_meshes.success")
    if os.path.exists(ems):
        successful += 1
        
    emj = os.path.join(r"D:\ig_pipeline\cad", s, r"artifacts\export_meshes.json")
    if os.path.exists(emj):
        tried += 1

print(f"Out of {len(sources)} sources")
print(f"Export meshes has been attempted for {tried}")
print(f"Was successful for {successful}")

Out of 1235 sources
Export meshes has been attempted for 1234
Was successful for 1232
